<a href="https://colab.research.google.com/github/LukeMedTkd/Image-Colorization/blob/main/dataset_managment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMAGE COLORIZATION - dataset managment
Questo notebook ha l'obbiettivo di presentare il dataset scelto per questo studio; ma ancora prima di presentare il dataset, è fondamentale comprendere qual è la domanda di ricerca sulla quale si incentra lo studio.

*È possibile “riportare in vita” vecchie foto in bianco e nero con metodi di apprendimento automatico profondo ?*

Da questa domanda, ha inizio l'indagine sull'applicazione del deep learning per questo specifico task.
In questo notebook, come annunciato prima, verrà presentato il dataset, verranno individuati tutti gli strumenti necessari per l'implementazione che seguirà

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
